# 0. Installing course dependencies

In [1]:
!pip install -r ../requirements.txt

  Using cached annoy-1.17.0.tar.gz (646 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached feedparser-6.0.8-py3-none-any.whl (81 kB)
  Using cached hnswlib-0.6.0.tar.gz (30 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (

In [6]:
!conda install -c conda-forge ffmpeg -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ok_boomer\Anaconda3\envs\ir

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ffmpeg-4.3.1               |       ha925a31_0        26.2 MB  conda-forge
    vc-14.2                    |       hb210afc_6          13 KB  conda-forge
    vs2015_runtime-14.29.30037 |       h902a5da_6         1.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        27.5 MB

The following NEW packages will be INSTALLED:

  ffmpeg             conda-forge/win-64::ffmpeg-4.3.1-ha925a31_0
  ucrt               conda-forge/win-64::ucrt-10.0.20348.0-h57928b3_0
  vc                 conda-forge/win-64::vc-14.2-hb210afc_6
  vs2015_runtime     conda-forge/win-64::vs2015_runtime-14.29.30037-h902a5da_6




vs2

Run the next cell if you want to download embedding model, but this is not required during this lab. You can do it later.

In [2]:
!python -m spacy download en_trf_distilbertbaseuncased_lg

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)
  Using cached wsproto-1.0.0-py3-none-any.whl (24 kB)
  Using cached h11-0.12.0-py3-none-any.whl (54 kB)
  Created wheel for annoy: filename=annoy-1.17.0-cp39-cp39-win_amd64.whl size=52536 sha256=d1adb014831748933bb64aafe046eed7dc2170e8c6f86b033f8eab95f0e3a83e
  Stored in directory: c:\users\ahuratus\appdata\local\pip\cache\wheels\71\d0\7a\354f18c65441aacd7e2806fe7e0d2f827da30e89683eb788bb
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=370d3d05a6c207b93fb5a48721408467bb275684885dd67355c9458f783d54e1
  Stored in directory: c:\users\ahuratus\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for hnswlib: filename=hnswlib-0.6.0-cp39-cp39-win_amd64.wh

# 1. Touching the Internet

Solve the following task. Download [this page](https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt), and save it to the file with the name derived from the URL. File with another URL should not be save into the file with this name. E.g. [this file](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/facts.txt).

Ref: [requests](https://docs.python-requests.org/en/latest/) library is cool.

In [5]:
import requests

url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
# TODO: download and save to a file
import hashlib
name = hashlib.sha256(url.encode()).hexdigest()
PATH='./SAVED'
site_to_save = requests.request('GET',url=url).text

import os 
if not os.path.exists(PATH):
    os.mkdir(PATH)
file = open(PATH+'/'+name+'.txt','w')
file.write(site_to_save)
file.close()

# 2. Parsing different formats

Most probably, if you meet something in Internet, this is: binary, plain text, XML, or json. XML also splits into xHTML, RSS, Atom, SOAP, XML-RPC, ... . Your task is to learn, how to process different formats.

## 2.1. JSON

In [the given file](http://sprotasov.ru/data/postnauka.txt) there is valid json. Parse it and print all video URLs, which have `computer science` tag. Use built-in features of `requests`, or just a `json` library ([ref](https://docs.python.org/3/library/json.html)).

In [17]:
import json
import requests

# TODO. Your code here
r=requests.get('http://sprotasov.ru/data/postnauka.txt')
data = json.loads(r.text.encode('utf-8').decode('utf-8-sig'))
results = []
for item in data:
    if 'computer science' in item['tags']:
        results.append(item['url'])
print(results)

['http://postnauka.ru/talks/31897', 'http://postnauka.ru/video/24306', 'http://postnauka.ru/faq/46974']


## 2.2. HTML

For a given StackExchange answer extract logins of the contributors (who asked and who answered) with votes. [bs4](https://beautiful-soup-4.readthedocs.io/en/latest/) will help you to do the job.

I can recommend to use CSS or XPath selectors. `div` elements with `post-layout` class represent answers. Inside there are `div` with `votecell` class stroring votes number and `div` with class `user-details` storing user info. My personal recommendation is to use `css selectors`, which are [documented here](https://beautiful-soup-4.readthedocs.io/en/latest/#css-selectors).

In [42]:
import requests
from bs4 import BeautifulSoup

url = f"https://math.stackexchange.com/questions/411486/understanding-the-singular-value-decomposition-svd"

# TODO. Your code here should parse HTML source page and find contributors of the repository.

r = requests.get(url)
content = r.text
soup = BeautifulSoup(content,'html')
votes = list(map(lambda x: int(x.text),soup.select('div.answer div.votecell div[data-value]')))
authors = list(map(lambda x: x.text,soup.select('div.answer div.post-signature .user-details a')))
for item in zip(authors,votes):
    print(item)


('Ittay Weiss', 16)
('Tomasz Bartkowiak', 8)
('Bart Vanderbeke', 4)
('Bart Vanderbeke', 3)
('hgfei', 2)
('littleO', 1)
('TheSHETTY-Paradise', 1)


# 2.3. RSS feed

A lot of information is already organized in typed XML documents. Podcasts are just RSS feed. Parse [the feed of this podcast](http://sprotasov.ru/podcast/rss.xml) and print out the time span between the first and the last episodes. Use [`feedparser` for this](https://waylonwalker.com/parsing-rss-python/).

In [12]:
import feedparser
rss = 'http://sprotasov.ru/podcast/rss.xml'
# feedparser.parse(rss) 

# TODO: complete the code to compute the time span of all the episodes.

# 3. Solving simple information retrieval task

According to the name, `information retrieval` is the discipline, which helps retrieves information (from unstructured sources). Thus, we will retrieve some information from [this news article](https://www.bbc.com/news/world-us-canada-59944889). Your task is to write a code, which will answer the question: **How many people die every day in the US waiting for a transplant?** Write flexible enough code. Test yourself by changing the link to [this one](https://www.americantransplantfoundation.org/about-transplant/facts-and-myths/).

In [13]:
import requests
url = 'https://www.bbc.com/news/world-us-canada-59944889'
question = 'How many people die every day in the US waiting for a transplant?'

# TODO. Impress me!